# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,blackmans bay,-43.0167,147.3167,15.96,53,31,0.45,AU,1728702351
1,1,uturoa,-16.7333,-151.4333,25.56,74,4,4.15,PF,1728702352
2,2,waitangi,-43.9535,-176.5597,10.46,97,100,4.47,NZ,1728702354
3,3,saint-pierre,-21.3393,55.4781,20.82,56,40,3.60,RE,1728702355
4,4,albany,42.6001,-73.9662,13.08,75,22,0.45,US,1728702341


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
        "Lng",
        "Lat",
        geo = True,
        tiles="OSM",
        size = "Humidity"
)
# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc[(city_data_df['Max Temp'] >= 22) & (city_data_df["Max Temp"] < 32)
                               & (city_data_df["Country"] == "US") & (city_data_df["Wind Speed"] < 5)]

# Drop any rows with null values
city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
12,12,holualoa,19.6228,-155.9522,23.26,94,95,3.35,US,1728702365
53,53,kapa'a,22.0752,-159.3190,27.18,84,0,2.57,US,1728702410
57,57,wailua homesteads,22.0669,-159.3780,26.44,84,0,2.57,US,1728702414
73,73,la'ie,21.6477,-157.9253,29.98,73,98,1.79,US,1728702433
92,92,lihue,21.9789,-159.3672,26.77,78,0,2.57,US,1728702454
95,95,hawaiian paradise park,19.5933,-154.9731,26.73,76,40,3.09,US,1728702457
309,309,clovis,36.8252,-119.7029,26.00,46,57,1.47,US,1728702487
363,363,safford,32.8340,-109.7076,28.51,13,0,0.00,US,1728702769
364,364,perryton,36.4000,-100.8027,23.00,25,47,4.83,US,1728702770
411,411,hilo,19.7297,-155.0900,26.85,76,40,3.09,US,1728702700


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# Code used to drop columns from df 
hotel_df = city_data_df.copy()
hotel_df = hotel_df.drop(columns=["City_ID", "Cloudiness", "Max Temp", "Date", "Wind Speed"])

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Lat,Lng,Humidity,Country,Hotel Name
12,holualoa,19.6228,-155.9522,94,US,
53,kapa'a,22.0752,-159.3190,84,US,
57,wailua homesteads,22.0669,-159.3780,84,US,
73,la'ie,21.6477,-157.9253,73,US,
92,lihue,21.9789,-159.3672,78,US,
95,hawaiian paradise park,19.5933,-154.9731,76,US,
309,clovis,36.8252,-119.7029,46,US,
363,safford,32.8340,-109.7076,13,US,
364,perryton,36.4000,-100.8027,25,US,
411,hilo,19.7297,-155.0900,76,US,


In [6]:
# Reposition Columns to match example
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity", "Hotel Name"]]

hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
12,holualoa,US,19.6228,-155.9522,94,
53,kapa'a,US,22.0752,-159.3190,84,
57,wailua homesteads,US,22.0669,-159.3780,84,
73,la'ie,US,21.6477,-157.9253,73,
92,lihue,US,21.9789,-159.3672,78,
95,hawaiian paradise park,US,19.5933,-154.9731,76,
309,clovis,US,36.8252,-119.7029,46,
363,safford,US,32.8340,-109.7076,13,
364,perryton,US,36.4000,-100.8027,25,
411,hilo,US,19.7297,-155.0900,76,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000

params = {"categories":"accommodation.hotel",
          "apiKey": geoapify_key,
          "limit": 20}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params = params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
holualoa - nearest hotel: Kona Hotel
kapa'a - nearest hotel: Pono Kai Resort
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
la'ie - nearest hotel: No hotel found
lihue - nearest hotel: Kauai Palms
hawaiian paradise park - nearest hotel: No hotel found
clovis - nearest hotel: BEST WESTERN Clovis Cole
safford - nearest hotel: Quality Inn And Suites
perryton - nearest hotel: Ambassador Inn
hilo - nearest hotel: Dolphin Bay Hotel
santa ana - nearest hotel: Holiday Inn Express
lahaina - nearest hotel: No hotel found
kailua-kona - nearest hotel: PACIFIC 19 Kona


,City,Country,Lat,Lng,Humidity,Hotel Name
12,holualoa,US,19.6228,-155.9522,94,Kona Hotel
53,kapa'a,US,22.0752,-159.3190,84,Pono Kai Resort
57,wailua homesteads,US,22.0669,-159.3780,84,Hilton Garden Inn Kauai Wailua Bay
73,la'ie,US,21.6477,-157.9253,73,No hotel found
92,lihue,US,21.9789,-159.3672,78,Kauai Palms
95,hawaiian paradise park,US,19.5933,-154.9731,76,No hotel found
309,clovis,US,36.8252,-119.7029,46,BEST WESTERN Clovis Cole
363,safford,US,32.8340,-109.7076,13,Quality Inn And Suites
364,perryton,US,36.4000,-100.8027,25,Ambassador Inn
411,hilo,US,19.7297,-155.0900,76,Dolphin Bay Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size = "Humidity",
    color = "City",
    tiles = "OSM",
    hover_cols = ["Hotel Name", "City"]
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name)